# 1. Viterbi가 왜 중요한가?

viterbi는 복호 성능이 뛰어나고, 동작 속도가 빠르다. 또한, 구현이 쉽고 복호 시간이 일정한 장점을 가지고 있는 것에 중요성을 가지고 있다.

출처 : http://www.ktword.co.kr/abbr_view.php?m_temp1=3195

# 2. Viterbi는 일상생활 어디에 적용되는가?


무선랜에서 사용하는 길쌈 부호를 해독하는데 사용하였으나 지금은 음성 인식, 음성 합성, 화자 구분, 키워드 검출, 전산언어학, 생물정보학 분야에서도 널리 활용되고 있다. 음성 관련에서는 자연어처리나 품사분석기에 특히 많이 이용된다.

출처 : https://ko.wikipedia.org/wiki/%EB%B9%84%ED%84%B0%EB%B9%84_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98

# 3. Viterbi를 활용한 알고리즘이나 패키지에는 무엇이 있는가?

viterbi-trellis 라이브러리가 있다. 이 라이브러리는 viterbi 알고리즘을 사용해서 만든 격자 모양 그래프를 통해 최적의 길 값을 계산한다.

https://pypi.org/project/viterbi-trellis/

In [2]:
pip install viterbi-trellis

Note: you may need to restart the kernel to use updated packages.


In [5]:
from viterbi_trellis import ViterbiTrellis

v = ViterbiTrellis([[2, 6, 4], [4, 6], [0, 2, 6]], lambda x: x / 2.0, lambda x, y: abs(y - x))
best_path = v.viterbi_best_path()

print(best_path)

[2, 0, 1]


# 4. Viterbi의 핵심 아이디어는 무엇인가?

score는 곧 확률이라 할 수 있다. 즉, 데이터들에 순서가 있을 때, 가장 높은 확률을 가진 확률들을 연결한다면, 그것은 path가 되는 것이다. 이 path를 만들 때 인지해야할 것은, path를 구성하는 데이터들(event)들 끼리는 서로가 연관이 있다는 것이다. 이 말은 과거의 evnet에 영향을 받는다는 소리이다. 데이터를 잘 정제하기 위해선 과거를 규정해야 하는데, 그러는 것은 힘들다. 그래서 고안된 것이 Markov property이다. 이것은 이벤트의 현재 혹은 바로 전 일만 보자는 것이다. 그렇게 해서 만든 model을 Hidden Markov Model(HMM)이라고 한다.

HMM과 같이 hidden variable을 포함하는 모델에선, hidden variables의 sequences를 결정하는 것이 문제이다. 이것을 decoding이라고 한다.

![Viterbi_markove_property](./images/Viterbi_markove_property.png)
요약하자면 풀고자하는 문제를 찾을 때, word가 들어갔을 때, tag의 확률값을 최소로 하는 tag sequences를 찾는 tag를 찾겠다는 뜻이다.

HMM을 요약해보면 이렇게도 만들 수 있다.
![viterbi_markov_model](./images/viterbi_markov_model.png)
P(w1 ^ n)이 전체 확률에 영향을 안 미치는 이유는, 모든 확률값들이 이미 P(w1^n)에게 영향을 받았기 때문에 나누는 의미가 없기 때문이다.

P(w1^n|t1^n)을 단순화 하는 것은, 모든 word와 tag를 찾는 것은 어렵기 때문에 timeslots로 쪼개어 단순 모델링을 하겠다는 뜻이다. 즉, P(w1^n|t1^n)을 p1 * p2 * ... pn으로 만들겠다는 뜻이다.

P(t1^n)단순화도, time slots로 나눈 Markov property를 이용해 단순화 시킨다.

Emission probability : 특정한 상황을 만들어낸 observation을 역으로 emission 측면에서 본 것

Transition probability : 과거의 상태에서 현재상태로 전이될 확률

viterbi 알고리즘의 장점 중 하나는 구현하기가 쉽다는 것이다.
![Viterbi_algorithm](./images/Viterbi_algorithm.png)

viterbi의 진행도를 살펴보자.
![viterbo_idea](./images/viterbi_idea.png)
time 단위로 divide & slot한다는 건, dynamic programming한다고 할 수 있다. 이는 이전 계산에다가 조금만 추가해서 다음 계산을 진행한다고 할 수 있다. 이전 값들을 다 곱했을 떄, 최대화 시키는 값 i를 찾는 것이다.

back-tracing이란 시점 t부터 1까지 거꾸로 탐색하겠다는 뜻이다. 이러한 검색을 하기 위해, 각 time step마다 현재값 i로 전이되는 최곳값 j(바로 전의 최곳값)에 대해 기록을 따로 해놓는다.